In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
warnings.filterwarnings("ignore") 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from sklearn.pipeline import Pipeline
from feature_engine.encoding import OrdinalEncoder, OneHotEncoder, WoEEncoder, RareLabelEncoder
from feature_engine.imputation import RandomSampleImputer, CategoricalImputer
from feature_engine.transformation import LogTransformer
from feature_engine.transformation import LogCpTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier

In [2]:
df = pd.read_csv('/Users/katana/Desktop/SBAnational.csv', low_memory=False)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/katana/Desktop/SBAnational.csv'

In [112]:
df.head(15)

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,84,4,2.0,0,0,1,0,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,60,2,2.0,0,0,1,0,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,180,7,1.0,0,0,1,0,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,60,2,1.0,0,0,1,0,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,240,14,1.0,7,7,1,0,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"
5,1000084002,"B&T SCREW MACHINE COMPANY, INC",PLAINVILLE,CT,6062,"TD BANK, NATIONAL ASSOCIATION",DE,332721,28-Feb-97,1997,120,19,1.0,0,0,1,0,N,N,NaN,30-Jun-97,"$517,000.00",$0.00,P I F,$0.00,"$517,000.00","$387,750.00"
6,1000093009,MIDDLE ATLANTIC SPORTS CO INC,UNION,NJ,7083,WELLS FARGO BANK NATL ASSOC,SD,0,2-Jun-80,1980,45,45,2.0,0,0,0,0,N,N,24-Jun-91,22-Jul-80,"$600,000.00",$0.00,CHGOFF,"$208,959.00","$600,000.00","$499,998.00"
7,1000094005,WEAVER PRODUCTS,SUMMERFIELD,FL,34491,REGIONS BANK,AL,811118,28-Feb-97,1997,84,1,2.0,0,0,1,0,N,Y,NaN,30-Jun-98,"$45,000.00",$0.00,P I F,$0.00,"$45,000.00","$36,000.00"
8,1000104006,TURTLE BEACH INN,PORT SAINT JOE,FL,32456,CENTENNIAL BANK,FL,721310,28-Feb-97,1997,297,2,2.0,0,0,1,0,N,N,NaN,31-Jul-97,"$305,000.00",$0.00,P I F,$0.00,"$305,000.00","$228,750.00"
9,1000124001,INTEXT BUILDING SYS LLC,GLASTONBURY,CT,6073,WEBSTER BANK NATL ASSOC,CT,0,28-Feb-97,1997,84,3,2.0,0,0,1,0,N,Y,NaN,30-Apr-97,"$70,000.00",$0.00,P I F,$0.00,"$70,000.00","$56,000.00"


In [113]:
def make_target(
    df: pd.DataFrame,
    source_col: str = "MIS_Status",
    target_col: str = "target",
    positive_class: str = "CHGOFF"
) -> pd.DataFrame:
    """
    Создаёт бинарный target для задачи кредитного дефолта.

    Parameters
    ----------
    df : pd.DataFrame
        Исходный DataFrame
    source_col : str
        Колонка с исходным статусом кредита
    target_col : str
        Название новой target-колонки
    positive_class : str
        Значение, которое считается дефолтом (1)

    Returns
    -------
    pd.DataFrame
        DataFrame с добавленной колонкой target
    """

    df = df.copy()

    # Нормализация текстовых значений
    df[source_col] = (
        df[source_col]
        .astype(str)
        .str.upper()
        .str.strip()
    )

    # Бинарный таргет
    df[target_col] = (df[source_col] == positive_class).astype(int)

    return df

In [114]:
df = make_target(df)

df["target"].value_counts(normalize=True)

target
0    0.824773
1    0.175227
Name: proportion, dtype: float64

In [115]:
def add_city_risk_group(
    df: pd.DataFrame,
    city_col: str = "City",
    target_col: str = "target",
    min_loans: int = 50,
    n_groups: int = 3,
    labels: list = None
) -> pd.DataFrame:
    """
    Добавляет категориальный признак CityRiskGroup
    на основе дефолтности по городам.

    Parameters
    ----------
    df : pd.DataFrame
        Исходный DataFrame
    city_col : str
        Колонка с названием города
    target_col : str
        Бинарный таргет (1 = дефолт)
    min_loans : int
        Минимальное число кредитов для учёта города
    n_groups : int
        Количество риск-групп (обычно 3)
    labels : list
        Названия групп риска

    Returns
    -------
    pd.DataFrame
        DataFrame с колонкой CityRiskGroup
    """

    if labels is None:
        labels = ['LOW_RISK', 'MEDIUM_RISK', 'HIGH_RISK']

    df = df.copy()

    # 1. Агрегация по городам
    city_stats = (
        df.groupby(city_col)
          .agg(
              TotalLoans=(target_col, 'count'),
              DefaultRate=(target_col, 'mean')
          )
          .reset_index()
    )

    # 2. Фильтр по минимальному количеству наблюдений
    city_stats = city_stats[city_stats['TotalLoans'] >= min_loans]

    # 3. Разбиение на риск-группы
    city_stats['CityRiskGroup'] = pd.qcut(
        city_stats['DefaultRate'],
        q=n_groups,
        labels=labels
    )

    # 4. Мерж обратно
    df = df.merge(
        city_stats[[city_col, 'CityRiskGroup']],
        on=city_col,
        how='left'
    )

    return df

In [116]:
df = add_city_risk_group(df)

In [117]:
def clean_money_columns(df, columns):
    '''
    Приводит значения в колонках с денежными выражениями к простым float.
    columns - список переменных
    '''
    for col in columns:
        if col not in df.columns:
            continue
        
        df[col] = (
            df[col]
            .astype(str)
            .str.strip()
            .str.replace(r"[^\d\.-]", "", regex=True)  
            .replace("", np.nan)                     
            .astype(float)
        )
    return df

In [118]:
money_cols = ["ChgOffPrinGr", "SBA_Appv", "GrAppv", "DisbursementGross"]

In [119]:
df = clean_money_columns(df, money_cols)

In [120]:
df['Bankrisk'] = df['GrAppv'] - df['SBA_Appv']

In [121]:
def add_sba_ratio(
    df: pd.DataFrame,
    sba_col: str = "SBA_Appv",
    gr_col: str = "GrAppv",
    ratio_col: str = "SBA_ratio"
) -> pd.DataFrame:
    """
    Создаёт признак отношения SBA_Appv / GrAppv.

    Обрабатывает деление на ноль, inf и NaN.

    Parameters
    ----------
    df : pd.DataFrame
        Исходный DataFrame
    sba_col : str
        Колонка с суммой SBA
    gr_col : str
        Колонка с общей суммой кредита
    ratio_col : str
        Название новой ratio-колонки

    Returns
    -------
    pd.DataFrame
        DataFrame с добавленной колонкой ratio
    """

    df = df.copy()

    df[ratio_col] = df[sba_col] / df[gr_col]

    # Убираем inf / -inf → NaN
    df[ratio_col] = df[ratio_col].replace([np.inf, -np.inf], np.nan)

    return df

In [122]:
df = add_sba_ratio(df)

In [123]:
def make_target(
    df: pd.DataFrame,
    source_col: str = "MIS_Status",
    target_col: str = "target",
    positive_class: str = "CHGOFF"
) -> pd.DataFrame:
    """
    Создаёт бинарный target для задачи кредитного дефолта.

    Parameters
    ----------
    df : pd.DataFrame
        Исходный DataFrame
    source_col : str
        Колонка с исходным статусом кредита
    target_col : str
        Название новой target-колонки
    positive_class : str
        Значение, которое считается дефолтом (1)

    Returns
    -------
    pd.DataFrame
        DataFrame с добавленной колонкой target
    """

    df = df.copy()

    # Нормализация текстовых значений
    df[source_col] = (
        df[source_col]
        .astype(str)
        .str.upper()
        .str.strip()
    )

    # Бинарный таргет
    df[target_col] = (df[source_col] == positive_class).astype(int)

    return df

In [124]:
df = make_target(df)

df["target"].value_counts(normalize=True)

target
0    0.824773
1    0.175227
Name: proportion, dtype: float64

In [165]:
df['target'].dtypes

dtype('int64')

In [125]:
var_to_drop = ['LoanNr_ChkDgt',
               'Name',
               'Zip',
               'BankState',
               'ApprovalDate',
               'ApprovalFY',
               'ChgOffPrinGr', 
               'ChgOffDate', 
               'DisbursementDate', 
               'DisbursementGross', 
               'SBA_Appv', 
               'BalanceGross', 
               'MIS_Status',
               'City'
            ]

In [126]:
X = df.drop('target', axis = 1)
y = df['target']

In [127]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size=0.3,
                                                   stratify=y,
                                                   random_state=45)

In [128]:
le_target = LabelEncoder()
y_train= le_target.fit_transform(y_train)
y_test = le_target.transform(y_test)

print(f" Target закодирован: {le_target.classes_}")

 Target закодирован: [0 1]


In [129]:
# Drop unnecessary vars

X_train = X_train.drop(var_to_drop, axis = 1)
X_test = X_test.drop(var_to_drop, axis = 1)

In [130]:
vars_with_na_cat = [var for var in X_train.columns if X_train[var].isnull().mean() > 0 and X_train[var].dtypes == 'O']
vars_with_na_cat

['State', 'Bank', 'RevLineCr', 'LowDoc']

In [131]:
vars_with_na_num = [var for var in X_train.columns if X_train[var].isnull().mean()>0 and X_train[var].dtypes !='O']
vars_with_na_num

['NewExist', 'CityRiskGroup']

In [132]:
cat_vars = [var for var in X_train.columns if X_train[var].dtypes =='O']
cat_vars

['State', 'Bank', 'RevLineCr', 'LowDoc']

In [159]:
num_vars = [var for var in X_train.columns if X_train[var].dtypes != 'O']
num_vars

['NAICS',
 'Term',
 'NoEmp',
 'NewExist',
 'CreateJob',
 'RetainedJob',
 'FranchiseCode',
 'UrbanRural',
 'GrAppv',
 'CityRiskGroup',
 'Bankrisk',
 'SBA_ratio']

In [133]:
cat_vars = ['State', 'Bank', 'RevLineCr', 'LowDoc', 'CityRiskGroup']

In [153]:
log_vars = [
    'NoEmp',
    'CreateJob',
    'RetainedJob'
]

In [158]:
print(X_train.columns)

Index(['State', 'Bank', 'NAICS', 'Term', 'NoEmp', 'NewExist', 'CreateJob',
       'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr', 'LowDoc',
       'GrAppv', 'CityRiskGroup', 'Bankrisk', 'SBA_ratio'],
      dtype='object')


In [160]:
xgb = Pipeline(steps = [
    
    ('imputer_cat', CategoricalImputer(
        variables=vars_with_na_cat,
        imputation_method='missing',
    )),
     
    ('imputer', RandomSampleImputer(
        variables=vars_with_na_num,
        random_state=42,
        seed = 'general'
    )),
    

    
    ('encoder', OrdinalEncoder(
        variables = cat_vars,
        encoding_method = 'arbitrary',
        unseen = "encode"
    )),


    ('transformer', LogCpTransformer(variables=log_vars)),

    
    ('classifier', XGBClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=25,
        subsample=0.8,
        colsample_bytree=0.8,
        min_child_weight=5,
        gamma=0.1,
        reg_alpha=1.0,
        reg_lambda=5.0,
        eval_metric='auc',
        enable_categorical=True, 
        tree_method='hist',
        #scale_pos_weight=50,
        random_state=42,
        n_jobs=-1
    ))
])

In [161]:
# Проверка для импьютера
print("Типы данных колонок в vars_with_na_cat:")
print(X_train[vars_with_na_cat].dtypes)

# Проверка для энкодера
print("\nТипы данных колонок в cat_vars:")
print(X_train[cat_vars].dtypes)

Типы данных колонок в vars_with_na_cat:
State        object
Bank         object
RevLineCr    object
LowDoc       object
dtype: object

Типы данных колонок в cat_vars:
State              object
Bank               object
RevLineCr          object
LowDoc             object
CityRiskGroup    category
dtype: object


In [162]:
xgb.fit(X_train, y_train)

,steps,"[('imputer_cat', ...), ('imputer', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,imputation_method,'missing'
,fill_value,'Missing'
,variables,"['State', 'Bank', ...]"
,return_object,False
,ignore_format,False
,variables,"['NewExist', 'CityRiskGroup']"
,random_state,42


In [163]:
# TRAIN
xgb_pred_train = xgb.predict(X_train)
xgb_proba_train = xgb.predict_proba(X_train)[:, 1]

# TEST
xgb_pred_test = xgb.predict(X_test)
xgb_proba_test = xgb.predict_proba(X_test)[:, 1]

In [164]:
print("\n=== TRAIN METRICS (XGBOOST) ===")
print("Confusion Matrix:")
print(confusion_matrix(y_train, xgb_pred_train))
print("\nClassification Report:")
print(classification_report(y_train, xgb_pred_train, digits=3))
print(f"ROC-AUC: {roc_auc_score(y_train, xgb_proba_train):.3f}")
print(f"F1: {f1_score(y_train, xgb_pred_train):.3f}")

print("\n=== TEST METRICS (XGBOOST) ===")
print("Confusion Matrix:")
print(confusion_matrix(y_test, xgb_pred_test))
print("\nClassification Report:")
print(classification_report(y_test, xgb_pred_test, digits=3))
print(f"ROC-AUC: {roc_auc_score(y_test, xgb_proba_test):.3f}")
print(f"F1: {f1_score(y_test, xgb_pred_test):.3f}")


=== TRAIN METRICS (XGBOOST) ===
Confusion Matrix:
[[513907   5217]
 [  7714 102576]]

Classification Report:
              precision    recall  f1-score   support

           0      0.985     0.990     0.988    519124
           1      0.952     0.930     0.941    110290

    accuracy                          0.979    629414
   macro avg      0.968     0.960     0.964    629414
weighted avg      0.979     0.979     0.979    629414

ROC-AUC: 0.996
F1: 0.941

=== TEST METRICS (XGBOOST) ===
Confusion Matrix:
[[217210   5272]
 [  7598  39670]]

Classification Report:
              precision    recall  f1-score   support

           0      0.966     0.976     0.971    222482
           1      0.883     0.839     0.860     47268

    accuracy                          0.952    269750
   macro avg      0.924     0.908     0.916    269750
weighted avg      0.952     0.952     0.952    269750

ROC-AUC: 0.980
F1: 0.860
